<a href="https://colab.research.google.com/github/charlessantarosa/dl-emotion-recognition-uea/blob/master/dl_emotion_recognition_uea.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div>
    <h1><strong>Dataset: Natural Human Face Images for Emotion Recognition</strong></h1>
    <h4>Link: <a href="https://www.kaggle.com/sudarshanvaidya/random-images-for-face-emotion-recognition">https://www.kaggle.com/sudarshanvaidya/random-images-for-face-emotion-recognition</a></h4>
    <div id="text">
        <p>
São 5.500 + imagens com 8 categorias de emoções - raiva, desprezo, nojo, medo, felicidade, neutralidade, tristeza e surpresa. Todas as imagens contêm rosto humano em tons de cinza (ou esboço). Cada imagem tem uma escala de cinza de 224 x 224 pixels no formato PNG.</p>
<p>As imagens são obtidas na Internet, onde estão disponíveis gratuitamente para download. Google, Unsplash, Flickr etc.</p>
    </div>
</div>

In [ ]:
import os
import cv2
from urllib.request import urlopen
import math
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

import seaborn as sns
from matplotlib import pyplot

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder

import tensorflow as tf
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense, GlobalAvgPool2D, GlobalMaxPool2D
from keras.preprocessing.image import load_img, img_to_array
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import plot_model

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D

from keras.utils import np_utils
from keras.utils.np_utils import to_categorical

In [ ]:
# Currenty Directory
PATH_ROOT = !pwd
PATH_ROOT

# Global Vars
DATASET_FILE = 'dl-emotion-recognition-uea.zip'
DATASET_GOOGLE_DRIVER_ID = '1J0xMHoQqC6zBibSxukK-Gshfb5kK-P1y'
DATASET_PATH = PATH_ROOT[0] + '/dataset/'

# Download Dataset.zip from GoogleDriver
!gdown --id $DATASET_GOOGLE_DRIVER_ID
!unzip -o $DATASET_FILE

# !ls -la $DATASET_PATH

# DATASET_PATH = '/home/charles/uea/deep-learning/datasets/dl-emotion-recognition-uea/'
# !ls -la $DATASET_PATH

In [ ]:
!ls -la $DATASET_PATH

In [ ]:
# Processing directory dataset

total_images = 0
classes = []
for _dir in os.listdir(DATASET_PATH):
    count = 0
    for f in os.listdir(DATASET_PATH + _dir + "/"):
        count += 1
        total_images += 1
        
    classes.append(_dir)
    print(f"{_dir} => {count} imagens")
    
print('\n----------------------------------------------')    
print(f"\nTotal: {total_images} imagens")
print('Total de classes: ', len(classes))
print('Classes: ', classes)
print('\n----------------------------------------------')    

In [ ]:
img_arr = np.empty(shape=(total_images, 48, 48, 3))
img_label = np.empty(shape=(total_images))
label_to_text = {}

i = 0
e = 0
for dir_ in os.listdir(DATASET_PATH):
    if dir_ in classes:
        label_to_text[e] = dir_
        for f in os.listdir(DATASET_PATH + dir_ + "/"):

            image = cv2.imread(DATASET_PATH + dir_ + "/" + f)
            img_arr[i] = cv2.resize(image, (48, 48))


            img_label[i] = e

            i += 1
        print(f"loaded all {dir_} images to numpy arrays")
        e += 1

img_arr.shape, img_label

In [ ]:
img_arr

In [ ]:
# Classes

label_to_text

In [ ]:
# Visualização Randômica de 2 imagens por classe.

fig = pyplot.figure(1, (8, 8))

idx = 0
for k in label_to_text:
    
    sample_indices = np.random.choice(np.where(img_label==k)[0], size=2, replace=False)
    sample_images = img_arr[sample_indices]
    
    for img in sample_images:
        idx += 1
        ax = pyplot.subplot(4, 4, idx)
        ax.imshow(img[:,:,0], cmap='gray')
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_title(label_to_text[k])
        pyplot.tight_layout()


## Processamento de imagens

In [ ]:
train_data = np.array(img_arr, dtype="float") / 255.0
# test_data = np.array(test_data, dtype="float") / 255.0

le = LabelEncoder()
train_label = le.fit_transform(img_label)
train_label = to_categorical(train_label)

X_train, X_test, y_train, y_test = train_test_split(train_data, train_label, test_size=0.20, random_state=42)
print(X_train, X_test, y_train, y_test)

In [ ]:
data_generator = ImageDataGenerator(
        rotation_range=20,
        zoom_range=0.15,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.15,
        horizontal_flip=True,
        fill_mode="nearest")

In [ ]:
model = Sequential()

model.add(Conv2D(30, (3, 3), input_shape=(48, 48, 3), activation='relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(15, (3, 3), input_shape=(3, 48, 48), activation='relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dropout(0.2))

model.add(Flatten())

model.add(Dense(512, activation='relu'))

model.add(Dense(256, activation='relu'))

model.add(Dense(128, activation='relu'))

model.add(Dense(64, activation='relu'))

model.add(Dense(num_classes, activation='softmax', name='predict'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

In [ ]:
model.fit(data_generator.flow(X_train, y_train),
                        # steps_per_epoch=len(X_train) / 128,
                        epochs=20, verbose=1,
                        validation_data=(X_test,y_test))

In [ ]:
scores = model.evaluate(X_test, y_test, verbose=0)
print("\nacc: %.2f%%" % (scores[1]*100))

## Validaçao do modelo


In [ ]:
image_url = 'https://raw.githubusercontent.com/charlessantarosa/dl-emotion-recognition-uea/main/e1.png'
req = urlopen(image_url)
image = np.asarray(bytearray(req.read()), dtype=np.uint8)
image = cv2.imdecode(image, -1)

plt.imshow(image, cmap='gray')

print(image.shape)
img = cv2.resize(image, (48, 48), interpolation = cv2.INTER_AREA)
img_pred = img.reshape(-1, 48, 48).astype('float32')

print(img_pred.shape)
# img_pred = img_pred.reshape(48, 48, 3).astype('float32')
# img_pred = img_pred/255.0



pred = model.predict(np.array(img_pred))
# pred_proba = model.predict_proba(img_pred)
# pred_proba = "%.2f%%" % (pred_proba[0][pred]*100)

# print(pred[0], " com confiança de ", pred_proba)